# Parsing articles

In [1]:
from dotenv import load_dotenv
import os
from tqdm import tqdm
import time
import json

from utils.parser import ParserConfig, Parser
from rag.retriever import RetrieverConfig
from rag.llm import LLMConfig
from rag.pipeline import LifeSpanGPT

load_dotenv()
LLAMA_CLOUD = os.getenv('LLAMA_CLOUD')
COHERE_TOKEN = os.getenv("COHERE_TOKEN")
OPENAI_TOKEN = os.getenv("OPENAI_TOKEN")
ROOT_PATH_ARTICLES = 'data' #folder with articles
ROOT_PATH_RESULTS = 'pipeline_results/test'#folder to save results

/Users/eriktadevosan/anaconda3/envs/dl/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in LLMConfig has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/eriktadevosan/LifespanGPT/rag/pipeline.py:7: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from rag.prompt_generator import PromptGeneratorConfig, PromptGenerator


In [ ]:
# for each article in ROOT_PATH_ARTICLES we parsing it into processed_data folder
for file in tqdm(os.listdir(ROOT_PATH_ARTICLES)):
    file_path = os.path.join(ROOT_PATH_ARTICLES,file)
    parser_config = ParserConfig(path_to_file=file_path,
                                llama_cloud_token=LLAMA_CLOUD,
                                instruction=None)
    parser = Parser(parser_config)
    parser.create_parser()
    parser.parse()

# Generating answers

In [2]:
# for each file in processed_data run pipeline
for file in tqdm(os.listdir("processed_data")):
    os.makedirs(ROOT_PATH_RESULTS, exist_ok=True)
    file_name = file.split(".md")[0]
    print(file_name)
    config = RetrieverConfig(
        file_path=f"processed_data/{file}",
        embeding_model="BAAI/bge-small-en",
        reranker_model="rerank-english-v3.0",
        chunk_size=15000,
        chunk_overlap=2000,
        COHERE_TOKEN=COHERE_TOKEN,
    )
    llm_config = LLMConfig(model_name="gpt-4o", temperature=0.0, api_key=OPENAI_TOKEN)
    pipeline = LifeSpanGPT(config, llm_config)
    answer = pipeline.run_pipeline()
    with open(f"{ROOT_PATH_RESULTS}/{file_name}.json", "w", encoding="utf-8") as f:
        json.dump(answer, f, indent=4)
    time.sleep(30)

  0%|          | 0/59 [00:00<?, ?it/s]

molecules25225339
Creating retriever


/Users/eriktadevosan/anaconda3/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/eriktadevosan/LifespanGPT/rag/retriever.py:56: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereRerank``.
  compressor = CohereRerank(


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/eriktadevosan/Library/Application Support/sagemaker/config.yaml
Creating llm
Generating prompt...


/Users/eriktadevosan/LifespanGPT/rag/qa.py:23: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(query)


What treatment or intervention or manipulation are used for mouse C57Bl/6 control male?
Generating prompt...
What are Lifespan or survival curve/results for mouse C57Bl/6 control male?
Generating prompt...


  0%|          | 0/59 [00:40<?, ?it/s]


KeyboardInterrupt: 